In [1]:
#! pip install sklearn

In [2]:
import fastai
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
from data.load_data import load_trustpilot_data
from data.text_pre import clean_text

In [3]:
df_trust = load_trustpilot_data()
df_trust = clean_text(df_trust, 'text')

In [4]:
#down sample
df0 = df_trust[df_trust['y'] == 0]
df0 = df0[:2000]

df1 = df_trust[df_trust['y'] == 1]
df1 = df1[:2000]

df_strat = pd.concat([df0.sample(df1.shape[0]),df1])

In [5]:
len(df_strat)

4000

In [6]:
df = pd.DataFrame({'label':df_strat.y, 'text':df_strat.text})

In [7]:
from sklearn.model_selection import train_test_split
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [8]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

In [9]:
# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [20]:
itos='itos'
lm_path='lm_best_with_opt' 
learn = language_model_learner(data_lm, pretrained_fnames=[lm_path,itos], drop_mult=0.7, arch=AWD_LSTM)

In [21]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,5.438405,4.854328,0.171389,00:03


In [22]:
learn.save_encoder('sentiment_enc')

In [23]:
learn = text_classifier_learner(data_clas, drop_mult=0.7, arch=AWD_LSTM)
learn.load_encoder('sentiment_enc')

In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,0.595373,0.443913,0.808750,00:07


In [25]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)

In [26]:
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,713,219
1,87,581


In [27]:
from sklearn.metrics import roc_auc_score, classification_report
print(roc_auc_score(targets, predictions))
print(classification_report(targets, predictions))

0.8087499999999999
              precision    recall  f1-score   support

           0       0.77      0.89      0.82       800
           1       0.87      0.73      0.79       800

   micro avg       0.81      0.81      0.81      1600
   macro avg       0.82      0.81      0.81      1600
weighted avg       0.82      0.81      0.81      1600

